In [32]:
import cv2
import numpy as np
from ultralytics import YOLO

model = YOLO(r"E:\다운로드\runs\detect\train\weights\best.pt")  # YOLO 모델 경로


img = cv2.imread(r"C:\Users\user\Documents\카카오톡 받은 파일\KakaoTalk_20250310_094742100_01.jpg",cv2.IMREAD_COLOR) 

results = model([img])

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    # result.show()  # display to screen
    # result.save(filename="result.jpg")  # saveto disk


0: 480x640 3 Labels, 210.8ms
Speed: 2.2ms preprocess, 210.8ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)


In [33]:
from sklearn.cluster import DBSCAN 

def get_object_detection_boxes(results):
    X = []

    for result in results:
        boxes = result.boxes  # box 객체

        if boxes is None or boxes.xyxy is None:
            continue

        for i in range(len(boxes)):
            xyxy = boxes.xyxy[i].cpu().numpy()
            x1, y1, x2, y2 = xyxy
            X.append([int(x1), int(y1), int(x2), int(y2)])

    return np.array(X)


In [34]:
import easyocr
import numpy as np

def extract_text_by_boxes_easyocr(image, boxes, reader=None):
    if reader is None:
        reader = easyocr.Reader(['ko'])

    texts = []
    for box in boxes:
        x1, y1, x2, y2 = map(int, box)
        cropped = image[y1:y2, x1:x2]

        if cropped.size == 0:
            texts.append(None)
            continue

        result = reader.readtext(cropped, detail=0, paragraph=False)

        if not result:
            texts.append(None)
        else:
            texts.append(result[0].strip())
            
            
        # print(result)    
    return np.array(texts).reshape(-1, 1)

In [35]:

def cluster_boxes(boxes, eps=800, min_samples=1):
    """
    boxes: List of bounding boxes [x1, y1, x2, y2]
    ocr_texts: OCR 결과 텍스트 리스트
    eps: DBSCAN의 거리 임계값 (같은 행으로 간주할 y축 거리)
    min_samples: 최소 샘플 수 (1로 하면 모든 점 포함 가능)
    
    return: 각 행에 대한 결과 
    """
    # 박스 중심 y좌표 계산
    y_centers = np.array([box[3] for box in boxes]).reshape(-1, 1)
    # print(y_centers)
    # DBSCAN으로 y축 클러스터링 → 같은 y좌표 = 같은 행
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(y_centers)
    labels = clustering.labels_  # 각 박스가 속한 행 번호

    return labels.reshape(-1,1)


In [ ]:
def row_ocr_clustering(image, results):
    boxes = get_object_detection_boxes(results)
    texts = extract_text_by_boxes_easyocr(image, boxes)
    row_labels = cluster_boxes(boxes)

    combined = np.concatenate([row_labels.reshape(-1, 1), texts.reshape(-1, 1), boxes ], axis=1)
    return combined  # shape: (n, 1[text], 4[bbox], 1[row])


In [ ]:
# n = 2
# y = clustering(X,n)


r = row_ocr_clustering(img,results)

print(r)




Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


[['0' '306 .' '1335' '549' '1439' '785']
 ['0' '306.953' '421' '521' '556' '764']
 ['0' '306.95' '211' '516' '311' '673']]
